# Synthetic Burst Testing

This notebook tests the scattering analysis pipeline using synthetic FRB data.

In [ ]:
import os
import sys
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt

notebook_dir = Path.cwd()
scattering_dir = notebook_dir.parent
flits_root = scattering_dir.parent
sys.path.insert(0, str(scattering_dir))
sys.path.insert(0, str(flits_root))

try:
    import chainconsumer, seaborn, emcee, arviz
except ImportError:
    print("Installing required packages...")
    os.system("pip install seaborn emcee chainconsumer arviz")

%load_ext autoreload
%autoreload 2

from scat_analysis.burstfit_pipeline import BurstPipeline
from scat_analysis.burstfit_corner import quick_chain_check, get_clean_samples, make_beautiful_corner_wide
from scat_analysis.burstfit import FRBParams, FRBModel
from scat_analysis.config_utils import TelescopeConfig

## 1. Create Synthetic Burst

In [ ]:
def create_synthetic_burst(n_time=2048, n_freq=256, snr=20.0, dm_true=350.0):
    from flits.common.constants import K_DM
    freqs = np.linspace(1500, 1300, n_freq)
    dt = 0.000131072
    times = np.arange(n_time) * dt
    t_center = n_time // 2 * dt
    width_s = 2.0 / 1000.0
    nu_ref = freqs[0]
    delays_s = K_DM * dm_true * (1/freqs**2 - 1/nu_ref**2) / 1000.0
    dynspec = np.zeros((n_time, n_freq), dtype=np.float64)
    for i_freq, (freq, delay) in enumerate(zip(freqs, delays_s)):
        t_arrival = t_center + delay
        profile = np.exp(-0.5 * ((times - t_arrival) / width_s)**2)
        tau_s = 0.5 * (freq / 1400.0)**-4.0 / 1000.0
        scatter_tail = np.exp(-times / tau_s) * (times >= 0)
        scattered = np.convolve(profile, scatter_tail, mode='same')
        dynspec[:, i_freq] = snr * scattered
    dynspec += np.random.randn(n_time, n_freq)
    return dynspec, freqs, dt

test_dynspec, test_freqs, test_dt = create_synthetic_burst(n_time=1024, n_freq=128, snr=15.0)
test_data_path = Path('../plots/test_burst_synthetic.npy')
test_data_path.parent.mkdir(parents=True, exist_ok=True)
np.save(test_data_path, test_dynspec)
print(f'Synthetic burst: shape={test_dynspec.shape}')

## 2. Freya-like Burst (CHIME)

In [ ]:
n_time, n_freq = 2048, 512
f_min, f_max = 400.19, 800.19
dt_ms = 2.56e-3
freqs_mhz = np.linspace(f_max, f_min, n_freq)
freqs_ghz = freqs_mhz / 1000.0
times_ms = np.arange(n_time) * dt_ms

c0, t0_ms, width_ms = 1.0, 2.6, 0.5
tau_1ghz_ms, alpha_scatter = 1.0, 4.0

dynspec_model = np.zeros((n_freq, n_time))
for i_freq, freq_ghz in enumerate(freqs_ghz):
    gauss = c0 * np.exp(-0.5 * ((times_ms - t0_ms) / width_ms) ** 2)
    tau_ms = tau_1ghz_ms * (freq_ghz / 1.0) ** (-alpha_scatter)
    scatter_times = times_ms - t0_ms
    scatter_kernel = np.where(scatter_times >= 0, np.exp(-scatter_times / tau_ms) / tau_ms, 0.0)
    dynspec_model[i_freq, :] = np.convolve(gauss, scatter_kernel * dt_ms, mode='same')

signal_peak = np.max(dynspec_model)
noise_std = signal_peak / 15.0
freya_synthetic = dynspec_model + np.random.randn(n_freq, n_time) * noise_std

freya_data_path = Path('../plots/test/freya_synthetic.npy')
freya_data_path.parent.mkdir(parents=True, exist_ok=True)
np.save(freya_data_path, freya_synthetic)
print(f'Freya synthetic: shape={freya_synthetic.shape}, tau_1ghz={tau_1ghz_ms} ms')

## 3. Casey-like Burst (DSA-110)

In [ ]:
n_time, n_freq = 2048, 256
f_min, f_max = 1280.0, 1530.0
dt_ms = 2.0e-3
freqs_mhz = np.linspace(f_max, f_min, n_freq)
freqs_ghz = freqs_mhz / 1000.0
times_ms = np.arange(n_time) * dt_ms

c0, t0_ms, width_ms = 1.0, 2.0, 0.3
tau_1ghz_ms, alpha_scatter = 0.227, 3.9

dynspec_casey = np.zeros((n_freq, n_time))
for i_freq, freq_ghz in enumerate(freqs_ghz):
    gauss = c0 * np.exp(-0.5 * ((times_ms - t0_ms) / width_ms) ** 2)
    tau_ms = tau_1ghz_ms * (freq_ghz / 1.0) ** (-alpha_scatter)
    scatter_times = times_ms - t0_ms
    scatter_kernel = np.where(scatter_times >= 0, np.exp(-scatter_times / tau_ms) / tau_ms, 0.0)
    dynspec_casey[i_freq, :] = np.convolve(gauss, scatter_kernel * dt_ms, mode='same')

signal_peak = np.max(dynspec_casey)
casey_synthetic = dynspec_casey + np.random.randn(n_freq, n_time) * (signal_peak / 20.0)

casey_data_path = Path('../plots/test/casey_synthetic.npy')
np.save(casey_data_path, casey_synthetic)
print(f'Casey synthetic: shape={casey_synthetic.shape}, tau_1ghz={tau_1ghz_ms} ms')
print(f'Note: Casey has ~15x less scattering than Freya!')

## Summary

| Test | tau_1ghz | Notes |
|------|----------|-------|
| Freya (CHIME) | ~1-3 ms | Strong scattering |
| Casey (DSA) | ~0.2 ms | Weak scattering |